In [1]:
import MySQLdb
import pandas as pd
import numpy as np
from datetime import datetime

In [2]:
hc_se = pd.read_table("/DAS_Storage1/aschoi/data/Drug_Repositioning/5_new_drug/new_predict/4_2_1_herbal_comp_gold_drug_se_sim_mat_predict.tsv", index_col=0)
hc_str = pd.read_table("/DAS_Storage1/aschoi/data/Drug_Repositioning/5_new_drug/new_predict/4_2_1_herbal_comp_gold_drug_se_sim_mat_predict.tsv", index_col=0)

In [3]:
hc_ids = hc_se.index.tolist()
print len(hc_ids)

100


In [4]:
#접속하기
db_handle = MySQLdb.connect('localhost', 'root','comsysbio', 'COCONUTv5')
#cur = db_handle.cursor(MySQLdb.cursors.DictCursor) # query를 통해 데이터를 얻을 때 python의 dict로 얻기 위한 방법
cursor = db_handle.cursor()

In [5]:
"{}".format(tuple(hc_ids))

'(5525, 1020, 732, 9210, 926, 4020, 86025, 70460, 82835, 79944, 79942, 79938, 79941, 1530, 1531, 79939, 153, 8019, 4286, 70319, 8885, 3761, 119, 139, 70410, 70661, 4521, 1878, 622, 543, 9471, 9926, 85183, 3005, 10195, 7874, 59654, 50263, 80081, 80082, 80097, 80164, 2969, 76198, 58454, 61885, 8739, 562, 77956, 614, 40553, 773, 65817, 71038, 70458, 5529, 1166, 71664, 8024, 837, 416, 79982, 7475, 83229, 89181, 760, 10086, 31743, 2682, 823, 79684, 845, 752, 70473, 7859, 78876, 1001, 666, 1574, 58258, 7996, 1167, 71205, 43066, 9670, 141, 243, 1331, 53060, 70973, 8012, 61516, 70924, 77084, 7638, 70692, 71009, 85310, 1343, 1529)'

In [6]:
# herbal compound와 관련있는 gene id 뽑기
query1 = "SELECT compID, geneID, reference FROM Compound_Gene where compID in {} and (organism=\"human\" or organism=\"homo sapiens\")".format(tuple(hc_ids))
cursor.execute(query1); #쿼리 날리는 거
results = cursor.fetchall() # cursor.fetchone()은 tuple, cursor.fetchall()은 list of tuples을 리턴한다.

In [7]:
np_results = np.array(results)

In [8]:
df_results = pd.DataFrame({'compID' : np_results[:, 0], 'geneID' : np_results[:, 1], 'reference' : np_results[:, 2]})

In [9]:
print len(df_results)
print len(df_results.drop_duplicates('compID')), len(df_results.drop_duplicates('geneID'))
df_results = df_results[['compID', 'geneID']].drop_duplicates()
print len(df_results)
df_results.head(1)

35808
66 8342
18525


,compID,geneID
0,119,283


In [10]:
# gene 정보가 없는 compound들
a = [int(x) for x in df_results['compID'].drop_duplicates().tolist()]
hc_se.index[~hc_se.index.isin(a)]

Int64Index([70460, 82835, 79944, 79941, 70319,  8885, 70410, 70661, 85183,
            59654, 50263, 80081, 76198, 58454, 61885, 40553, 65817, 71038,
            70458, 71664, 89181, 31743, 79684, 70473, 71205, 53060, 70973,
            61516, 70924, 77084,  7638, 70692, 71009, 85310],
           dtype='int64')

In [11]:
hc_ids2 = hc_se.index[hc_se.index.isin(a)].tolist()

In [19]:
# geneId에 해당하는 entrezID와 uniprotID 뽑기
gene_ids = [int(x) for x in df_results.geneID.drop_duplicates().tolist()]
query2 = "SELECT geneID, entrezID, uniprotID FROM Gene where geneID in {}".format(tuple(gene_ids))
cursor.execute(query2); #쿼리 날리는 거
results2 = cursor.fetchall() # cursor.fetchone()은 tuple, cursor.fetchall()은 list of tuples을 리턴한다.

In [31]:
np_results2 = np.array(results2)
df_results2 = pd.DataFrame({'geneID' : np_results2[:,0], 'entrezID' : np_results2[:,1], 'uniprotID':np_results2[:,2]})

In [32]:
print len(df_results2)
print len(df_results2.drop_duplicates('entrezID')), len(df_results2.drop_duplicates('geneID')) # entrez는 null 인것이 사라짐
print len(df_results2)
df_results2.head(1)

8342
8325 8342
8342


,entrezID,geneID,uniprotID
0,2147,1,P00734 / A0N064|A8IM47|C8BKD1|E7FAN5|G1NEM6|G1...


In [33]:
# uniprot ID를 row당 하나 오게 바꾸기
aa = df_results2['uniprotID'].str.split('/').apply(pd.Series, 1).stack()
aa.index = aa.index.droplevel(-1)
aa.name = 'uniprotID'
temp = df_results2.copy()
del temp['uniprotID']
bb = temp.join(aa)
cc = bb['uniprotID'].str.split('|').apply(pd.Series, 1).stack()
cc.index = cc.index.droplevel(-1)
cc.name = 'uniprotID'
temp = bb.copy()
del temp['uniprotID']
dd = temp.join(cc)

In [60]:
df_results3 = dd.drop_duplicates()

In [61]:
df_results4 = df_results3[df_results3.uniprotID != "null"]

In [63]:
df_results4.head(1)

,entrezID,geneID,uniprotID
0,2147,1,P00734


In [64]:
df_results.head(1)

,compID,geneID
0,119,283


In [67]:
df_results4.head(1)

,entrezID,geneID,uniprotID
0,2147,1,P00734


In [68]:
merged = pd.merge(left = df_results, right = df_results4, how='inner')

In [69]:
print len(df_results), len(df_results4)
print len(merged), len(merged.drop_duplicates())
print len(merged.drop_duplicates('compID')), len(merged.drop_duplicates('entrezID')), len(merged.drop_duplicates('uniprotID'))
merged.head(1)

18525 248617
633301 633301
66 8255 248346


,compID,geneID,entrezID,uniprotID
0,119,283,4846,P29474


In [70]:
# herbal compounds의 gene id를 uniprot에서 입력으로 넣어서 가져온 protein 정보
hc_seq_raw = pd.read_table("/home/share/aschoi/nas/users/asolchoi/data/Drug_Repositioning/5_new_drug/new_predict/hc_seq_raw.tab")

In [71]:
hc_seq_raw.head(1)

,Entry,Entry name,Sequence,Length,Organism ID
0,A0A0A0MTA6,A0A0A0MTA6_HUMAN,MGNLKSVAQEPGPPCGLGLGLGLGLCGKQGPATPAPEPSRAPASLL...,614,9606


In [72]:
hc_seq = pd.merge(left = merged, right=hc_seq_raw, left_on = 'uniprotID', right_on='Entry', how='inner')

In [73]:
print len(hc_seq)
hc_seq.head(1)

26400


,compID,geneID,entrezID,uniprotID,Entry,Entry name,Sequence,Length,Organism ID
0,119,283,4846,E7ESA7,E7ESA7,E7ESA7_HUMAN,MFTYICNHIKYATNRGNLRSAITVFPQRCPGRGDFRIWNSQLVRYA...,997,9606


In [74]:
sub = hc_seq[['entrezID', 'uniprotID', 'Sequence']].drop_duplicates()

In [ ]:
sub.sort_values(by=['entrezID', 'Sequence']).to_csv("/home/share/aschoi/nas/users/asolchoi/data/Drug_Repositioning/entrez_uniprot.tsv", sep='\t')

In [76]:
sub = sub.sort_values(by=['entrezID', 'Sequence'])

In [75]:
ori_sub = pd.read_table("/home/share/aschoi/nas/users/asolchoi/data/Drug_Repositioning/entrez_uniprot.tsv", index_col=0)

In [79]:
id_len = [len(x) for x in sub.uniprotID]

In [80]:
sub['id_length'] = id_len

In [81]:
sub.head()

,entrezID,uniprotID,Sequence,id_length
1464,10,P11245,MDIEAYFERIGYKNSRNKLDLETLTDILEHQIRAVPFENLNMHCGQ...,6
14488,100,F5GWI4,MAQTPAFDKPKVELHVHLDGSIKPETILYYGRRRGIALPANTAEGL...,6
14493,100,P00813,MAQTPAFDKPKVELHVHLDGSIKPETILYYGRRRGIALPANTAEGL...,6
17881,1000,P19022,MCRIAGALRTLLPLLAALLQASVEASGEIALCKTGFPEDVYSAVLS...,6
16346,10003,Q9Y3Q0,MAESRGRLYLWMCLAAALASFLMGFMVGWFIKPLKETTTSVRYHQS...,6


In [82]:
sub_b = sub.sort_values(['entrezID', 'id_length']).drop_duplicates(['entrezID'])

In [83]:
sub_b.head(1)

,entrezID,uniprotID,Sequence,id_length
1464,10,P11245,MDIEAYFERIGYKNSRNKLDLETLTDILEHQIRAVPFENLNMHCGQ...,6


In [86]:
print len(sub_b.entrezID.drop_duplicates()), len(sub_b.uniprotID.drop_duplicates()), len(sub_b[~sub_b.uniprotID.isin(tarSeq.Target)].drop_duplicates('uniprotID'))

7999 7977 7528


In [87]:
len(merged[merged.uniprotID.isin(sub_b.uniprotID)].drop_duplicates('uniprotID'))

7977

In [88]:
# gold protein을 제외한 hc만의 protein --> 이후 gold protein 내용도 합쳐야함. 하지만 계산 빨리 하기 위해서 지금은 제외함.
except_gold_protein = sub_b[~sub_b.uniprotID.isin(tarSeq.Target)].drop_duplicates('uniprotID')[['uniprotID', 'Sequence']]

In [91]:
#except_gold_protein.to_csv("/DAS_Storage1/aschoi/data/Drug_Repositioning/5_new_drug/new_predict/4_3_herbal_compounds_target_except_gold.tsv", sep='\t', index=False)
#except_gold_protein.to_csv("/home/share/aschoi/nas/users/asolchoi/data/Drug_Repositioning/5_new_drug/new_predict/4_3_herbal_compounds_target_except_gold.tsv", sep='\t', index=False)
except_gold_protein.to_csv("/home/share/aschoi/nas/users/asolchoi/data/Drug_Repositioning/5_new_drug/new_predict/4_3_herbal_compounds_target_except_gold_new.tsv", sep='\t', index=False)
except_gold_protein.to_csv("/DAS_Storage1/aschoi/data/Drug_Repositioning/5_new_drug/new_predict/4_3_herbal_compounds_target_except_gold_new.tsv", sep='\t', index=False)

In [469]:
#tarSeq[['Target', 'Sequence']].drop_duplicates().to_csv("/home/share/aschoi/nas/users/asolchoi/data/Drug_Repositioning/5_new_drug/new_predict/gold_target.tsv", sep='\t', index=False)

In [107]:
tarSim = pd.read_excel("/DAS_Storage1/aschoi/data/Drug_Repositioning/5_new_drug/new_predict/similarity_protein_gold_indep.xlsx", header=None)

In [85]:
tarSeq = pd.read_table("/DAS_Storage1/aschoi/data/Drug_Repositioning/5_new_drug/new_predict/drug_target_seq.tsv")

In [108]:
tarSim.head(1)

,0,1,2,3,4,5,6,7,8,9,...,2299,2300,2301,2302,2303,2304,2305,2306,2307,2308
0,1.0,0.007301,0.007301,0.00921,0.00921,0.015303,0.015303,0.015303,0.015303,0.013242,...,0.012339,0.013388,0.005891,0.005685,0.013741,0.011477,0.012288,0.010557,0.015177,0.012273


In [109]:
gold_indep_target = pd.read_table("/DAS_Storage1/aschoi/data/Drug_Repositioning/5_new_drug/new_predict/drug_target_seq.tsv")

In [111]:
tarSim.columns = gold_indep_target.Entry.tolist()
tarSim.index = gold_indep_target.Entry.tolist()

In [112]:
print tarSim.shape
print tarSeq.shape

(2309, 2309)
(2309, 5)


In [113]:
print len(tarSeq.Target.drop_duplicates())

670


In [92]:
gold_target = pd.read_table("/home/share/aschoi/nas/users/asolchoi/data/Drug_Repositioning/5_new_drug/new_predict/gold_target.tsv")

In [50]:
herb_target = pd.read_table("/home/share/aschoi/nas/users/asolchoi/data/Drug_Repositioning/5_new_drug/new_predict/4_3_herbal_compounds_target_except_gold.tsv")

In [95]:
hc_tar_ex = pd.read_table("/DAS_Storage1/aschoi/data/Drug_Repositioning/5_new_drug/new_predict/similarity_protein_hc_gold_index.tsv", index_col=0)

In [97]:
hc_tar_ex.shape

(7590, 670)

In [131]:
hc_tar_ex_new = hc_tar_ex.filter(items = except_gold_protein.uniprotID, axis=0)

In [117]:
ext_gold_tar_id = sub_b[sub_b.uniprotID.isin(tarSeq.Target)].drop_duplicates('uniprotID').uniprotID.tolist()

In [121]:
tarSim.head(1)

,P35228,Q9P0X4,Q9P0X4,P00519,P00519,P23219,P23219,P23219,P23219,P21728,...,Q9UGJ0,Q9UGI9,Q14571,Q14573,Q14123,P51160,P16499,P35913,Q9Y698,Q92806
P35228,1.0,0.007301,0.007301,0.00921,0.00921,0.015303,0.015303,0.015303,0.015303,0.013242,...,0.012339,0.013388,0.005891,0.005685,0.013741,0.011477,0.012288,0.010557,0.015177,0.012273


In [129]:
tmp1 = tarSim[~tarSim.index.duplicated()]
tmp2 = tmp1.loc[:, ~tmp1.columns.duplicated()]
ext_gold_tar = tmp2.filter(items = ext_gold_tar_id, axis=0)

In [134]:
hc_tar_sim = pd.concat([hc_tar_ex_new, ext_gold_tar])

In [141]:
hc_tar_sim.head(1)

,P35228,Q9P0X4,P00519,P23219,P21728,P51168,P20309,O43246,Q07869,Q12809,...,Q9UGJ0,Q9UGI9,Q14571,Q14573,Q14123,P51160,P16499,P35913,Q9Y698,Q92806
P11245,0.014274,0.009387,0.015436,0.020695,0.020867,0.021093,0.026532,0.01722,0.028348,0.014849,...,0.034546,0.024622,0.011882,0.010611,0.015694,0.015634,0.0215,0.021294,0.017281,0.022545


In [140]:
hc_seq.head(1)

,compID,geneID,entrezID,uniprotID,Entry,Entry name,Sequence,Length,Organism ID
0,119,283,4846,E7ESA7,E7ESA7,E7ESA7_HUMAN,MFTYICNHIKYATNRGNLRSAITVFPQRCPGRGDFRIWNSQLVRYA...,997,9606


In [147]:
gold_tar_sim = pd.read_table("/DAS_Storage1/aschoi/data/Drug_Repositioning/5_new_drug/new_predict/1_drug_drug_target_sim_mat.tsv", index_col=0)

In [148]:
gold_tar_sim.head(1)

,DB00114,DB00126,DB00136,DB00153,DB00162,DB00165,DB00175,DB00176,DB00177,DB00178,...,DB01576,DB01577,DB01590,DB01623,DB02546,DB03904,DB04272,DB04844,DB04861,DB09026
DB00114,1.0,0.042848,0.03069,0.03069,0.057624,0.038125,0.019586,0.026963,0.027824,0.015547,...,0.03124,0.046066,0.016044,0.032962,0.031915,0.039085,0.061324,0.032962,0.033651,0.029795


In [151]:
hc_seq.head(1)

,compID,geneID,entrezID,uniprotID,Entry,Entry name,Sequence,Length,Organism ID
0,119,283,4846,E7ESA7,E7ESA7,E7ESA7_HUMAN,MFTYICNHIKYATNRGNLRSAITVFPQRCPGRGDFRIWNSQLVRYA...,997,9606


In [161]:
hc = hc_seq.compID.drop_duplicates().tolist()
gd = gold_tar_sim.index.tolist()

In [163]:
df = pd.DataFrame(columns=gd, index=hc)
df.fillna(0.0, inplace=True)

In [164]:
for i in hc:
    targets1 = hc_seq[hc_seq.compID == i].Entry.tolist()
    for j in gd:
        targets2 = gold_indep_target[gold_indep_target.DrugBank_ID == j].Target.tolist()
        
        max_sim = hc_tar_sim.filter(items = targets1, axis =0).filter(items=targets2).max().max()
        
        df.set_value(i, j, max_sim)

KeyboardInterrupt: 